Esse script é o produto do REO4 - Trabalho Final - da disciplina de Avanços em Genética e Melhoramento de Plantas (PGM848) do Período letivo 2020/1. Para mais informações, acesse: https://github.com/VQCarneiro/Visao-Computacional-no-Melhoramento-de-Plantas

                                UNIVERSIDADE FEDERAL DE LAVRAS - UFLA
                                    DEPARTAMENTO DE BIOLOGIA - DBI
                        PROGRAMA DE PÓS-GRADUÇÃO EM GENÉTICA E MELHORAMENTO DE PLANTAS
                DISCIPLINA PGM848 - AVANÇOS CIENTÍFICOS EM GENÉTICA E MELHORAMENTO DE PLANTAS
                                DOCENTE DSc. VINÍCIUS QUINTÃO CARNEIRO
                                 DISCENTE ERIK MICAEL DA SILVA SOUZA
                                      REO4 - TRABALHO FINAL

                            ROTINA DE PROCESSAMENTO DE IMAGENS DE MORANGO

Obs.: Importar as bibliotecas necessárias para resolução desta lista de exercícios: numpy, pil, glob, os, opencv (cv2), matplotlib, pandas & scikit-image (funções).

In [3]:
import cv2 
import PIL
from PIL import Image
import numpy as np 
import cv2 
import os
import glob
from matplotlib import pyplot as plt
import pandas as pd
from skimage.measure import label, regionprops
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy import ndimage

**Passos:**

**1) Definindo o diretório de trabalho:**

In [4]:
dire = os.getcwd()
print(dire)

C:\Users\Erik-Souza\Documents\final\fotos morangos


**2) Criando a Função *Resize_Img(img=imagem, mywidth=número final de colunas, x=número da imagem, dire=diretório)* para redimensionamento das imagens de morango:**

In [6]:
def Resize_Img(img, mywidth, x, dire):
    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    path = r"{}\imagens reduzidas".format(dire)
    if not os.path.exists(path):
        os.mkdir(path)
    img.save('{}\img{}.png'.format(path, x))

**3) Carregando as imagens da pasta *Fotos-morango*, aplicando a Função Resize_Img(), criando e salvando as imagens redimensionadas na pasta *imagens reduzidas*:**

In [ ]:
'''
img_dir = r"{}\Fotos-morango".format(dire)
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
data = [] 
for f1 in files: 
    img = Image.open(f1) 
    data.append(img)

j = 0
for i in data:
    j = j + 1
    Resize_Img(i, 1200, j, dire)
'''

**4) Criando a Função *proc_img(img=imagem, j=número da imagem, Med=Tamanho do Kernel para suavização via mediana, d=Tamanho do Kernel para dilatação do thresh, e=Tamanho do Kernel para erosão do thresh, dire=diretório)* para processamento das imagens de morango:**

In [9]:
def proc_img(img, j, Med, d, e, dire):
    path = r"{}\imagens segmentadas".format(dire)
    if not os.path.exists(path):
        os.mkdir(path)
    img_bgr = img[100:600, 100:1100]
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    B, G, R = cv2.split(img_bgr)
    # suavizando a imagem antes de aplicar o thresholding
    R1 = cv2.medianBlur(R, Med)
    (L, mask) = cv2.threshold(R1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Thresholding da imagem
    kernel = np.ones((d, d), np.uint8)  # Transformação morfológica do thresh
    dilatation = cv2.dilate(mask, kernel)
    kernel2 = np.ones((e, e), np.uint8)
    erosion = cv2.erode(dilatation, kernel2)
    # segmentação da imagem
    img_seg = cv2.bitwise_and(img_bgr, img_bgr, mask=erosion)
    cv2.imwrite('{}\segmenta {}.png'.format(path, j), img_seg)

**5) Carregando as imagens da pasta *imagens reduzidas*, aplicando a Função proc_img(), criando e salvando as imagens processadas na pasta *imagens segmentadas*:**

In [10]:
img_dir = r"{}\imagens reduzidas".format(dire)
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
data1 = []
for f1 in files: 
    img = cv2.imread(f1,1) 
    data1.append(img) 

j = 0
for i in data1:
    j = j + 1
    proc_img(i, j, 17, 9, 7, dire)

**6) Criando a Função *obt_ext(img_bgr=imagem, j=número da imagem, Med=Tamanho do Kernel para suavização via mediana, d=Tamanho do Kernel para dilatação do thresh, e=Tamanho do Kernel para erosão do thresh, dire=diretório)* para extração dos objetos das imagens de morango:**

In [11]:
def obt_ext(img_bgr, j, med, d, e, dire):
    path = r"{}\objetos".format(dire)
    if not os.path.exists(path):
        os.mkdir(path)
    path1 = r"{}\dados objetos".format(dire)
    if not os.path.exists(path1):
        os.mkdir(path1)
    img_bgr = img[100:600, 100:1100]
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    B, G, R = cv2.split(img_bgr)
    # suavizando a imagem antes de aplicar o thresholding
    R1 = cv2.medianBlur(R, med)
    (L, mask) = cv2.threshold(R1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Thresholding da imagem
    kernel = np.ones((d, d), np.uint8)  # Transformação morfológica do thresh
    dilatation = cv2.dilate(mask, kernel)
    kernel2 = np.ones((e, e), np.uint8)
    erosion = cv2.erode(dilatation, kernel2)
    img_seg = cv2.bitwise_and(img_bgr, img_bgr, mask=erosion)
    _, cnts, h = cv2.findContours(erosion, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    dimen = []
    for (i, c) in enumerate(cnts):

            (x, y, w, h) = cv2.boundingRect(c)
            obj = mask[y:y+h, x:x+w]
            obj_bgr = img_seg[y:y+h, x:x+w]
            obj_rgb = cv2.cvtColor(obj_bgr,cv2.COLOR_BGR2RGB)
            cv2.imwrite('{}/Morango {} - Objeto {}.png'.format(path,j,i+1), obj_bgr)
            
            area = cv2.contourArea(c)
            razao = round((h/w), 2)
            perim = round(cv2.arcLength(c, True), 2)
            tam_ret = np.shape(obj)

            regiao = regionprops(obj)
            rm = round(regiao[0].minor_axis_length, 2)
            rmai = round(regiao[0].major_axis_length, 2)
            cen = regiao[0].centroid
            
            min_val_r, max_val_r, min_loc_r, max_loc_r = cv2.minMaxLoc(obj_rgb[:,:,0], mask=obj)
            med_val_r = cv2.mean(obj_rgb[:,:,0], mask=obj)

            min_val_g, max_val_g, min_loc_g, max_loc_g = cv2.minMaxLoc(obj_rgb[:, :, 1], mask=obj)
            med_val_g = cv2.mean(obj_rgb[:,:,1], mask=obj)

            min_val_b, max_val_b, min_loc_b, max_loc_b = cv2.minMaxLoc(obj_rgb[:, :, 2], mask=obj)
            med_val_b = cv2.mean(obj_rgb[:,:,2], mask=obj)
            
            dimen += [[str(i + 1), str(h), str(w), str(area), str(razao), str(perim), str(tam_ret), str(rm), str(rmai), str(cen), str(min_val_r), str(max_val_r), str(min_loc_r), str(max_loc_r),str(min_val_b), str(max_val_b), str(min_loc_b), str(max_loc_b), str(min_val_g), str(max_val_g), str(min_loc_g), str(max_loc_g)]]
            
            dados_obj = pd.DataFrame(dimen)
            dados_obj = dados_obj.rename(columns={0: 'Objeto', 1: 'Comprimento', 2: 'Largura', 3: 'Area', 4: 'Razao', 5: 'Perimetro', 6: 'Tamanho', 7:'Eixo Menor', 8:'Eixo Maior', 9:'Centroide', 10:'Max R', 11:'Min R', 12:'Loc Max R', 13:'Loc Min R', 14: 'Max G', 15:'Min G', 16:'Loc Max G', 17:'Loc Min G', 18:'Max B', 19:'Min B', 20:'Loc Max B', 21:'Loc Min B'})
            dados_obj.to_csv('{}\medidas imagem {}.csv'.format(path1,j), index=False)

**7) Carregando as imagens da pasta *imagens segmentadas*, aplicando a Função obt_ext(), criando e salvando os objetos processadas na pasta *objetos*. Os dados de cada objeto serão armazenados em uma planilha no formato .csv e armazenados na pasta *dados objetos*:**

In [12]:
img_dir = r"{}\imagens segmentadas".format(dire)
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
data2 = []
for f1 in files: 
    img = cv2.imread(f1,1) 
    data2.append(img) 

j = 0
for i in data2:
    j = j + 1
    obt_ext(i, j, 17 , 9, 7, dire)

Obs.: Os dados dos objetos/morangos foram unificados manual em um planilha excel e salvo em um arquivo txt nomeado *dados.morango* para realização das análises de variância e correlações.